In [1]:
import pandas as pd
import numpy as np
import folium
import requests
import json
import pprint
import datetime as dt
import re

In [2]:
room_data = pd.read_csv('room.csv',encoding='CP949')
room_data.head()

,건물명(상호명),건물위치,주택유형구분,세대수(객실수),건축연도
0,NaN,전라북도 전주시 덕진구 고랑동 819-5,다세대주택(3세대),3,19920810
1,NaN,전라북도 전주시 덕진구 금암동 1569-6,다세대주택(3세대),6,19891206
2,NaN,전라북도 전주시 덕진구 금암동 1588-23,다세대주택(3세대),3,19890803
3,휴엔하임 주상복합 아파트,전라북도 전주시 덕진구 금암동 1639-1,"공동주택(아파트), 업무시설(오피스텔), 근린생활시설",30,20161128
4,NaN,전라북도 전주시 덕진구 금암동 525-83,다세대주택,4,19891121


In [3]:
room_data['세대수(객실수)'].value_counts()

3      1332
10     1066
11      788
9       643
4       616
5       576
8       542
2       497
6       423
12      349
7       302
15      296
13      246
18      201
1       192
0       169
14      134
16      129
19      111
17       91
30        4
20        3
54        2
159       2
56        2
24        1
48        1
288       1
182       1
110       1
73        1
172       1
44        1
34        1
826       1
77        1
53        1
27        1
51        1
167       1
276       1
28        1
191       1
Name: 세대수(객실수), dtype: int64

In [4]:
room_data['주택유형구분'].value_counts()

단독주택                      3751
단독주택,제1종근린생활시설             679
다가구주택                      494
단독주택(다가구주택)                417
단독주택(3가구)                  196
                          ... 
단독주택(다가구주택-9가구)              1
소매점.사무소.다가구주택                1
다가구주택(10가구),제1종근생(소매점)       1
단독주택(다가구주택(17가구))            1
다가구단독주택(17가구)                1
Name: 주택유형구분, Length: 741, dtype: int64

In [5]:
room_data.describe()

,세대수(객실수)
count,8734.000000
mean,8.135791
std,11.724445
min,0.000000
25%,3.000000
50%,8.000000
75%,11.000000
max,826.000000


In [6]:
p = re.compile(r'\d+')
room_data['건축연도'] = room_data['건축연도'].map(lambda x: x if p.search(x) else '0')
room_data.drop(room_data[room_data['건축연도'].map(lambda x:int(x[:4])) < 1945].index, inplace=True) #해방 전 옛 건물들 제거
room_data.drop(room_data[room_data['세대수(객실수)']<1].index, inplace=True) #세대수가 0인 것 제거
room_data['const_year_point'] = 5-(dt.datetime.today().year-room_data['건축연도'].map(lambda x: int(x[:4])))//5

room_data.describe()

,세대수(객실수),const_year_point
count,8555.000000,8555.000000
mean,8.301344,2.547165
std,11.789038,1.858123
min,1.000000,-10.000000
25%,4.000000,1.000000
50%,8.000000,3.000000
75%,11.000000,4.000000
max,826.000000,5.000000


In [7]:
# TODO
# 건물 위치를 참조하여 folium에 좌표 찍기
# 건축 연도를 참조하여 room_data에 신축 / 준신축 / 보통 / 재건축필요 등의 등급으로 분류

# room_data['건물위치']
# key = "7F8A26C5-7C0A-318B-A521-FE1AA35942DB"
# arr = []
# f = open("./loc_all.txt", 'w')
# f.close()
# import time
# for i in room_data['건물위치']:
#     f = open("./loc_all.txt", 'a')
#     location = i.strip()
#     response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=PARCEL")
#     if 'result' in response.json()['response']:
#         x= float(response.json()['response']['result']['point']['x'])
#         y = float(response.json()['response']['result']['point']['y'])
#         arr.append([float(x),float(y)])
#         f.write(str(x))
#         f.write(',')
#         f.write(str(y))
#         f.write(',')
#         f.write(location)
#         f.write('\n')
#         f.close()
#     else:
#         print(location, response.json())
#     time.sleep(0.05)

In [8]:
f = open('./loc.txt', 'r')
lines = f.readlines()
map1 = folium.Map(location=[35.83001,127.125001], zoom_start=13)
arr = []
for line in lines:
    lat,long = map(float, line.split(','))
    lat = round(lat,2)
    long = round(long,2)
    if [lat,long] not in arr:
        arr.append([lat,long])

for content in arr:
    lat,long = content
    folium.Marker([long, lat], popup=f'{long}-{lat}의 원룸', icon=folium.Icon(color='red',icon='info-sign')).add_to(map1)
    folium.Circle([long, lat], radius=100, color='#3186cc', fill_color='#3186cc', popup=f'{long}-{lat}의 원룸 반경 100M').add_to(map1)
    folium.Circle([long, lat], radius=500, color='#ebe134', fill_color='#ebe134', popup=f'{long}-{lat}의 원룸 반경 500m').add_to(map1)
map1.save('map1.html')

In [9]:
from bs4 import BeautifulSoup

url = "http://openapi.jeonju.go.kr/jeonjubus/openApi/traffic/bus_location2_stopnm_common.do?serviceKey=cjE984BWrGFH3DFA2kLDoNx7j950nWjp6Ajz0CBaEABY5arYHY1C80RNBw6ptmayEV6B0zzpOfAbB73miZ4gnA%3D%3D&searchFld=searchNM&searchNm=%EC%95%88%EA%B3%A8"
response = requests.get(url)
data = BeautifulSoup(response.text, 'xml')
stopX = data('stopX')
stopY = data('stopY')
stopKname = data('stopKname')
stopId = data('stopStandardid')
bus_stop_arr = []

for i in range(len(stopX)):
    x=float(stopX[i].text)
    y=float(stopY[i].text)
    name = stopKname[i].text
    stop_id = int(str(stopId[i].text).strip())
    if 35.78<y and y<35.94 and 127.06<x and x<127.19:
        bus_stop_arr.append([x,y,name,stop_id])
bus_stop_arr.sort(key=lambda x:x[0])
print(len(bus_stop_arr))
for element in bus_stop_arr:
    if (35.78<element[1] and element[1]<35.94) and (127.06<element[0] and element[0]<127.19):
        folium.Marker([element[1], element[0]], popup=f'위도 : {element[1]}-경도 : {element[0]}:{element[2]}', icon=folium.Icon(color='green',icon='info-sign')).add_to(map1)
        
map1.save('map_bus.html')

1408


In [11]:
### 반지름 내의 원에 정류장 좌표가 몇 개 있는지 계산

for content in arr[-2:-1]:
    print(content)
    station_count = 0
    bus_count = 0
    content_lat = content[0]
    content_long = content[1]
    near_bus_stop = []
    bus_route_arr = []
    for bus_stop in bus_stop_arr:
        if bus_stop[0]>content_lat-0.005 and bus_stop[0]<content_lat+0.005 and bus_stop[1]>content_long-0.005 and bus_stop[1]<content_long+0.005:
            near_bus_stop.append(bus_stop)
    for bus_stop in near_bus_stop:
        if ((content_lat-bus_stop[0])**2+(content_long-bus_stop[1])**2)**(1/2) <= ((content_lat-(content_lat-0.0055))**2+(content_long-(content_long-0.0044))**2)**(1/2):
            bus_stop_id = bus_stop[3]
            url = f'http://openapi.jeonju.go.kr/jeonjubus/openApi/traffic/general_busstop_common.do?serviceKey=cjE984BWrGFH3DFA2kLDoNx7j950nWjp6Ajz0CBaEABY5arYHY1C80RNBw6ptmayEV6B0zzpOfAbB73miZ4gnA%3D%3D&stopStandardid={bus_stop_id}'
            response = requests.get(url)
            data = BeautifulSoup(response.text, 'xml')
            all_route=data('brtStdId')
            this_bus_route_count=0
            for route in range(len(all_route)):
                route_id = all_route[route]
                if route_id not in bus_route_arr:
                    bus_route_arr.append(route_id)
                    this_bus_route_count+=1
#             if this_bus_route_count>=1:
            print(bus_stop[2],'-(',bus_stop[3],')',this_bus_route_count,'대')
    print('총 :',len(bus_route_arr))

[127.11, 35.8]
효천초등학교 -( 305032500 ) 8 대
효천대방노블랜드 -( 305032497 ) 0 대
효천우미린2차아파트 -( 305032501 ) 0 대
효천리버클래스아파트 -( 305032498 ) 0 대
효천우미린2차아파트 -( 305032502 ) 0 대
우전초교 -( 305100488 ) 0 대
우전초교 -( 305100485 ) 0 대
삼천주공6단지 -( 305100382 ) 2 대
삼천주공6단지 -( 305100381 ) 0 대
삼천호반리젠시빌 -( 305100412 ) 0 대
삼천호반리젠시빌 -( 305100422 ) 0 대
삼천신일강변아파트 -( 305100408 ) 2 대
삼천신일강변아파트 -( 305100421 ) 0 대
총 : 12


In [43]:
## CCTV
cctv_url =  'http://openapi.jeonju.go.kr/rest/cctv/getCctvList?serviceKey=cjE984BWrGFH3DFA2kLDoNx7j950nWjp6Ajz0CBaEABY5arYHY1C80RNBw6ptmayEV6B0zzpOfAbB73miZ4gnA%3D%3D&pageSize=1400&startPage=1'
cctv_response = requests.get(cctv_url)
cctv_data = BeautifulSoup(cctv_response.text, 'xml')
cctv_posX = cctv_data('cctvPosx')
cctv_posY = cctv_data('cctvPosy')
dong_addr = cctv_data('dongAddr')
cctv_arr = []
for i in range(len(cctv_posX)):
    if len(cctv_posX[i].text) > 3 and len(cctv_posY[i].text)>3:
        x_position = round(float(cctv_posX[i].text),3)
        y_position = round(float(cctv_posY[i].text),3)
        if [x_position,y_position] not in cctv_arr:
            cctv_arr.append([x_position,y_position])
           
        
cctv_map = folium.Map(location=[35.83001,127.125001], zoom_start=13)
            
## Police_box
# dukjin_jigudae_addr = ['전주시 덕진구 안덕원로 173', '전주시 덕진구 진버들4길 10', '전주시 덕진구 권삼득로 396']
# dukjin_pachulso_addr = ['전주시 덕진구 진북1길 19', '전주시 덕진구 동부대로 871', '전주시 덕진구 쪽구름로 82', '전주시 덕진구 신복로 58', 
#                        '전주시 덕진구 솔내6길 23', '전주시 덕진구 송천로 50', '전주시 덕진구 동부대로 688', '전주시 덕진구 출판로 92']
# dukjin_police_addr = ['전북 전주시 덕진구 온고을로 299']
# wansan_jigudae_addr = ['전주시 완산구 대동로 95', '전주시 완산구 성지산로 24', '전주시 완산구 서원로 295', '전주시 완산구 새터로46', 
#                        '전주시 완산구 용머리로 125', '전주시 완산구 마전들로 1', '전주시 완산구 평화로 159']
# wansan_pachulso_addr = ['전주시 완산구 서학 3길 46']
# wansan_police_addr = ['전주시 완산구 전라감영로 66']      
# key = "7F8A26C5-7C0A-318B-A521-FE1AA35942DB"
# police_addr = [dukjin_jigudae_addr, dukjin_pachulso_addr, dukjin_police_addr, wansan_jigudae_addr, wansan_pachulso_addr, wansan_police_addr]
# police_loc = []
# for sub_addr in police_addr:
#     for location in sub_addr:
#         response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=ROAD")
#         if 'result' in response.json()['response']:
#             x= float(response.json()['response']['result']['point']['x'])
#             y = float(response.json()['response']['result']['point']['y'])
#             police_loc.append([x,y])

police_loc = [[127.146461566, 35.834468712], [127.16380635, 35.835774879], [127.120456415, 35.848296399], [127.132636253, 35.826902578], [127.149387537, 35.863971253], [127.076131218, 35.872743759], [127.107977884, 35.856099777], [127.123538425, 35.858744294], [127.118451299, 35.857844477], [127.160431827, 35.850066374], [127.062692384, 35.839356441], [127.092696544, 35.84296123], [127.146036617, 35.823575242], [127.121085454, 35.79975135], [127.126785309, 35.812004112], [127.111398444, 35.835411819], [127.123993749, 35.806270762], [127.096152619, 35.821300798], [127.135674461, 35.791003554], [127.151890494, 35.807513991], [127.146370384, 35.815267116]]

for content in arr:
    cctv_count = 0
    police_count = 0
    content_lat = content[0]
    content_long = content[1]
    
    for cctv in cctv_arr:
        if ((content_lat-cctv[0])**2+(content_long-cctv[1])**2)**(1/2) <= ((content_lat-(content_lat-0.0055))**2+(content_long-(content_long-0.0055))**2)**(1/2):
            cctv_count+=1
    for police in police_loc:
        if ((content_lat-police[0])**2+(content_long-police[1])**2)**(1/2) <= ((content_lat-(content_lat-0.0055))**2+(content_long-(content_long-0.0055))**2)**(1/2):
            police_count+=1    
    lat,long = content
    
    iframe = folium.IFrame(f'''{long}-{lat}의 원룸 반경 500m: 
                            <br> CCTV 갯수 : {cctv_count}
                            <br> 파출소 갯수 : {police_count}
                            <br> 버스노선 수 :
                            <br> 근처 버스정류장 수 :
                            ''')
    popup = folium.Popup(iframe,
                     min_width=500,
                     max_width=500)
    
    folium.Marker([long, lat], popup=f'{long}-{lat}의 원룸', icon=folium.Icon(color='red',icon='info-sign')).add_to(cctv_map)
    folium.Circle([long, lat], radius=100, color='#3186cc', fill_color='#3186cc', popup=f'{long}-{lat}의 원룸 반경 100M').add_to(cctv_map)
    folium.Circle([long, lat], radius=500, color='#ebe134', fill_color='#ebe134', popup=popup).add_to(cctv_map)
            
# for cctv in cctv_arr:
#     folium.Marker([cctv[1], cctv[0]], popup=f'위도 : {cctv[1]}-경도 : {cctv[0]}', icon=folium.Icon(color='black',icon='warning-sign')).add_to(cctv_map)
cctv_map.save('map_cctv.html')

[[127.146461566, 35.834468712], [127.16380635, 35.835774879], [127.120456415, 35.848296399], [127.132636253, 35.826902578], [127.149387537, 35.863971253], [127.076131218, 35.872743759], [127.107977884, 35.856099777], [127.123538425, 35.858744294], [127.118451299, 35.857844477], [127.160431827, 35.850066374], [127.062692384, 35.839356441], [127.092696544, 35.84296123], [127.146036617, 35.823575242], [127.121085454, 35.79975135], [127.126785309, 35.812004112], [127.111398444, 35.835411819], [127.123993749, 35.806270762], [127.096152619, 35.821300798], [127.135674461, 35.791003554], [127.151890494, 35.807513991], [127.146370384, 35.815267116]]
